# Verosimilitud de una variable normal bivariada

Graficamos las funciones de verosimilitud y de costo de una variable normal bivariada. Este ejemplo corresponde a un ajuste de datos de cuadrados mínimos lineal que vamos a realizar más adelante en el curso. 

Valor medido de la variable normal bivariada: 

In [ ]:
import numpy as np
x_measured = np.array([2.119, -0.727])

Errores y coefficiente de correlación conocidos:

In [ ]:
x_sigma = np.array([0.00491, 0.0411])
correlation_coefficient = 0.67

Construyo la matriz de covarianza

In [ ]:
import danatools
covariance_matrix = danatools.covariance_matrix_2d(x_sigma[0], x_sigma[1], correlation_coefficient)

El estimador de máxima verosimilitud es igual al dato medido:

In [ ]:
mu_mle = x_measured

## Verosimilitud

La función de verosimilitud tiene como argumento el parámetro vectorial 2D $\mu = (\mu_1, \mu_2)$. Cubro una región del espacio del parámetro $\mu$ alrededor del dato observado. Fuera de esta región la función de verosimilitud es casi 0. 

In [ ]:
nsigma = 3
mu1_min =  x_measured[0] - nsigma * x_sigma[0]
mu1_max =  x_measured[0] + nsigma * x_sigma[0]
mu1 = np.linspace(mu1_min, mu1_max, num=50)

In [ ]:
mu2_min =  x_measured[1] - nsigma * x_sigma[1]
mu2_max =  x_measured[1] + nsigma * x_sigma[1]
mu2 = np.linspace(mu2_min, mu2_max, num=50)

Definimos una malla en el espacio de $\mu$ para graficar la verosimilitud con matplotlib 

In [ ]:
mu1_mesh, mu2_mesh = np.meshgrid(mu1, mu2)
mu_mesh = np.dstack((mu1_mesh, mu2_mesh))
mu_mesh

Reusamos la PDF normal multivariada para la verosimilitud intercambiando la variable aleatoria $x$ y el parámetro $\mu$: 

In [ ]:
from scipy.stats import multivariate_normal
likelihood = multivariate_normal.pdf(x=mu_mesh, mean=x_measured, cov=covariance_matrix)
likelihood.shape

Con este truco evitamos escribir la verosimilitud explícitamente. Esto es posible para la PDF normal porque la variable aleatoria $x$ y el parámetro $\mu$ son simétricos. La PDF toma el mesh de $\mu$ como argumento y retorna un mesh de las mismas dimensiones con los valores de la verosimilitud correspondiente a cada punto en el espacio de $\mu$.

Con las mesh construidas, ploteamos la verosimilitud:

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm
fig = plt.figure(figsize=(8, 6.5))
ax = fig.subplots(subplot_kw={"projection": "3d"})
ax.set_xlabel("$\mu_1$")
ax.set_ylabel("$\mu_2$")
ax.set_zlabel("Likelihood")
ax.plot_surface(mu1_mesh, mu2_mesh, likelihood, cmap=cm.viridis)

## Función de costo

En la práctica es más habitual plotear las superficies de nivel en 2D en lugar de hacer una figura en 3D como la anterior. Como las superficies de nivel de la verosimilitud y la función de costo tienen la misma forma, y además como la función de costo es más simple, vamos a usar esta función para las plotear las superficies de nivel. Más adelante vamos a ver que las superficies de nivel además dan información sobre los errores de los parámetros.

Construímos la función de costo en 2D:

In [ ]:
cost_mesh = danatools.normal_cost_2d(mu_mesh, x_measured, covariance_matrix)
cost_mesh.shape

Como trabajar con un mesh en la función de costo conlleva alguna complejidad, usamos una función de la librería danatools para hacerlo.

Graficamos las superficies de nivel de la función de costo

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("$\mu_1$")
ax.set_ylabel("$\mu_2$")
ax.plot(mu_mle[0], mu_mle[1], marker='o', ls='', label="$\hat{\mu}$")
sigma_levels = np.arange(0, nsigma)
ax.contour(mu1_mesh, mu2_mesh, cost_mesh, levels=sigma_levels**2, linestyles='dashed')
ax.legend()

Dibujamos la posición del estimador de máxima verosimilitud y, en analogía con la PDF normal multivariada, marcamos las supeficies nsigma. 

Como para una variable normal bivariada, las superficies de nivel son elipses, como opción alternativa podemos dibujar la elipses directamente.

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel("$\mu_1$")
ax.set_ylabel("$\mu_2$")
ax.plot(mu_mle[0], mu_mle[1], marker='o', ls='', label="$\hat{\mu}$")
ax.plot(*danatools.get_ellipse(mu_mle, covariance_matrix, nsigma=1), ls='--')
ax.plot(*danatools.get_ellipse(mu_mle, covariance_matrix, nsigma=2), ls='--')
ax.legend()
fig.tight_layout()
# fig.savefig("verosimilitud_bivariada.svg")

En esta demo mostramos como plotear la función de verosimilitud y de costo para una variable normal bivariada. En el camino señalamos algunos detalles a tener en cuenta y mostramos como hacerlo aprovechando algunas librerías disponibles.